# Optically dark short GRBs

**Caden Gobat**, The George Washington University

## Imports/loading/setup

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, importlib, seaborn as sns, astropy.units as u
from scipy import interpolate
from src.utilities import new_since_Fong
from src.xrt import XRT_lightcurve, get_photonIndex, get_temporalIndex, get_columnDensity
from asymmetric_uncertainty import a_u

### Load legacy and new data

In [ ]:
sGRBs = pd.read_csv("./products/Swift_sGRB_catalog.csv")
new_sGRBs = new_since_Fong(sGRBs) # Fong et al. (2015) has data up to March 2015, i.e. GRB 150301A

BetaXData = pd.read_csv("./data/BetaXData.csv", header=None, names=["GRB","Beta_X","Beta_X_pos","Beta_X_neg"])
BetaXData["GRB"] = [entry.split("-")[-1] for entry in BetaXData["GRB"]]
BetaXData["Beta_X"] *= -1

OpticalData = pd.read_csv("./data/OpticalData.csv", header=None, names=["GRB","Time","Observatory","Instrument","Filter","Exposure","F_o","e_F_o"])
OpticalData["GRB"] = [entry.split("-")[-1] for entry in OpticalData["GRB"]]
OpticalData["Time"] *= 60*60 # hours to seconds

filters = pd.read_csv("./data/FilterInfo.csv", header=None, names=["Observatory","Instrument","Filter","Wavelength","Frequency"])
OpticalData = pd.merge(OpticalData,filters,how="left",on=["Observatory","Instrument","Filter"])

XRayData = pd.read_csv("./data/XRayData.csv", header=None, names=["GRB","Time","Exposure","F_x","e_F_x"])

xrt_data = pd.read_csv("./products/Swift_XRT_lightcurves.csv")

## X-ray data processing

### Flux conversions

$$ F_\text{x} = \int_{0.3\text{ keV}}^{10\text{ keV}} F(E)\ dE = \int_{7.254\cdot10^{16}\text{ Hz}}^{2.418\cdot10^{18}\text{ Hz}} F_\nu\ d\nu = A \int_{7.254\cdot10^{16}\text{ Hz}}^{2.418\cdot10^{18}\text{ Hz}} \nu^{-\beta}\ d\nu $$
$$ F_\text{x} = \left.\begin{cases} A\frac{\nu^{1-\beta}}{1-\beta}, & \beta \neq 1 \\ A\ln(\nu), & \beta = 1 \end{cases}\right\}_{7.254\cdot10^{16}\text{ Hz}}^{2.418\cdot10^{18}\text{ Hz}} $$

In [ ]:
for i,row in xrt_data.iterrows():
    grb_id = row["GRB"]
    if grb_id not in sGRBs["GRB"].values:
        continue
    beta = a_u(float(sGRBs.loc[sGRBs["GRB"]==grb_id,"Beta_X"]),
               float(sGRBs.loc[sGRBs["GRB"]==grb_id,"Beta_X_pos"]/1.645), # 90% conf to 1-sigma
               float(sGRBs.loc[sGRBs["GRB"]==grb_id,"Beta_X_neg"]/1.645)) # 90% conf to 1-sigma
    B = beta.value
    Fx = a_u(row["Flux"], row["Fluxpos"], np.abs(row["Fluxneg"]))
    if B == 1:
        integral = np.log(10) - np.log(0.3)
    else:
        integral = (10**(1-B) - 0.3**(1-B))/(1-B)

    log_mean_energy = 10**np.mean((np.log10(0.3),np.log10(10))) # halfway between the endpoints in log space

    dfdF = (log_mean_energy**(-B))/integral
    dfdB = -np.log(log_mean_energy)*log_mean_energy**(-B)*Fx.value/integral #- log_mean_energy**(-B)*(1-B)*Fx.value*(np.log(0.3)*0.3**(1-B) - np.log(10)*10**(1-B))/((10**(1-B)-0.3**(1-B))**2) - log_mean_energy**(-B)*Fx.value/(10**(1-B)-0.3**(1-B))
    pos_err = np.sqrt(dfdF**2*Fx.plus**2 + dfdB**2*beta.plus**2)
    neg_err = np.sqrt(dfdF**2*Fx.minus**2 + dfdB**2*beta.minus**2)
    result = Fx.value*log_mean_energy**(-B)/integral
    spectral_flux = a_u(result,pos_err,neg_err) # erg/s/cm^2/keV
    spectral_flux *= 1e23/241797944177033445 # convert to Jy
    xrt_data.loc[i,"SpecFlux"] = spectral_flux

### Fong, et al. (2015)'s data

In [ ]:
added_from_Fong = []
for grb_id in XRayData["GRB"].unique(): # add David's old data in the same format
    GRB = grb_id[3:]
    lightcurve = XRayData.loc[XRayData["GRB"]==grb_id,:]
    if GRB not in xrt_data["GRB"].values:
        added_from_Fong.append(GRB)
        for i in lightcurve.index:
            t = lightcurve.loc[i,"Time"]
            tpos = tneg = lightcurve.loc[i,"Exposure"]/2
            flux = lightcurve.loc[i,"F_x"]/1e6 # uJy to Jy
            if lightcurve.loc[i,"e_F_x"] != 0:
                fluxpos = fluxneg = lightcurve.loc[i,"e_F_x"]/1e6
            else:
                fluxpos = 0
                fluxneg = np.inf
            newrow = [dict(zip(xrt_data.columns,
                               [GRB,t,tpos,tneg,np.nan,np.nan,np.nan,
                                a_u(flux,fluxpos,fluxneg)]))]
            xrt_data = pd.concat([xrt_data, pd.DataFrame(newrow)], ignore_index=True)
            
xrt_data.sort_values(by=["GRB","Time"], ascending=[False,True], inplace=True)

In [ ]:
print("Finished xray.")

## Optical data processing

### Load and preprocess data

In [ ]:
new_optical = pd.read_excel("./data/newData.xlsx")

for col in ["GRB","TriggerNumber","Observatory","Instrument","Source","E(B-V)"]:
    for i in new_optical.index:
        if pd.isna(new_optical.loc[i,col]):
            new_optical.loc[i,col] = new_optical.loc[i-1,col] # deal with merged Excel cells
new_optical["Magnitude"] = pd.to_numeric(new_optical["Magnitude"],errors="coerce")
new_optical.dropna(subset=["Magnitude"],inplace=True)

rastinejad = pd.read_csv("./data/Rastinejad_Table1.csv")
for i in rastinejad.index:
    info = {}
    info["GRB"] = rastinejad.loc[i,"GRB"]
    if "/" in rastinejad.loc[i,"Telescope/Instrument"]:
        info["Observatory"] = rastinejad.loc[i,"Telescope/Instrument"].split("/")[0]
        info["Instrument"] = rastinejad.loc[i,"Telescope/Instrument"].split("/")[1]
    else:
        info["Observatory"] = rastinejad.loc[i,"Telescope/Instrument"]
    info["Filter"] = rastinejad.loc[i,"Filter"]
    info["Time (s)"] = float(rastinejad.loc[i,"dt [sec]"])
    if ">" in rastinejad.loc[i,"Magnitude"]:
        info["Magnitude"] = float(rastinejad.loc[i,"Magnitude"].replace(">",""))
        info["Mag error"] = "3-sigma"
    elif "+/-" in rastinejad.loc[i,"Magnitude"]:
        info["Magnitude"] = float(rastinejad.loc[i,"Magnitude"].split(" +/- ")[0])
        info["Mag error"] = float(rastinejad.loc[i,"Magnitude"].split(" +/- ")[1])
    info["λ_eff"] = float(rastinejad.loc[i,"Wavelength"])
    info["Source"] = "Rastinejad"
    info["E(B-V)"] = float(rastinejad.loc[i,"E(B-V)"])
    new_optical = pd.concat([new_optical, pd.DataFrame([info])], ignore_index=True)

*Swift*-UVOT magnitudes are natively reported in the Vega system. Here I standardize the filter names and apply the appropriate conversions to put them in AB.

In [ ]:
UVOT_conversions = {'V': -0.01, 'B': -0.13, 'U': 1.02, 'UVW1': 1.51, 'UVM2': 1.69, 'UVW2': 1.73, 'White': 0.8}

for i in new_optical.loc[new_optical["Observatory"]=="Swift"].index:
    filt = new_optical.loc[i,"Filter"]
    if filt in UVOT_conversions.keys() or pd.isna(filt):
        pass
    elif filt.upper() in UVOT_conversions.keys():
        new_optical.loc[i,"Filter"] = filt.upper()
    elif 'wh' in filt.lower():
        new_optical.loc[i,"Filter"] = "White"
    elif filt[-3:].upper() == "_FC":
        new_optical.loc[i,"Filter"] = filt[:-3].title()
    elif 'm2' in filt.lower():
        new_optical.loc[i,"Filter"] = "UVM2"
    elif 'w1' in filt.lower():
        new_optical.loc[i,"Filter"] = "UVW1"
    elif 'w2' in filt.lower():
        new_optical.loc[i,"Filter"] = "UVW2"
        
    new_optical.loc[i,"Magnitude"] += UVOT_conversions[new_optical.loc[i,"Filter"]]

In [ ]:
new_optical["mag_w_err"] = [a_u(mag,err,err) if isinstance(err,(float,int)) else a_u(mag,np.inf,0) for mag,err in new_optical[["Magnitude","Mag error"]].values]

### Extinction correction and flux conversion

$ A_b = R_b \cdot E_{B-V} $ for an arbitrary band $b$

In [ ]:
RbTable = pd.read_csv("./data/Rb.csv") # Table 6 from Schlafly & Finkbeiner (2011)
RbTable.drop([37,55,61,73],axis=0,inplace=True) # smoothing

Rb = interpolate.interp1d(RbTable["lambda_eff"],RbTable["R_b"],fill_value="extrapolate") # function that takes a wavelength [Ang] and returns the corresponding R_b value

new_optical["Extinction"] = Rb(new_optical["λ_eff"])*new_optical["E(B-V)"]

new_optical["Flux (Jy)"] = 3631*10**(-(new_optical["mag_w_err"]-new_optical["Extinction"])/2.5) # AB mag = 0 at f_nu = 3631 Jy

### Fong, et al. (2015)'s optical data

In [ ]:
all_optical = new_optical.copy()

for i in OpticalData.index:
    GRB = OpticalData.loc[i,"GRB"]
    obs = OpticalData.loc[i,"Observatory"]
    inst = OpticalData.loc[i,"Instrument"]
    filt = OpticalData.loc[i,"Filter"]
    time = OpticalData.loc[i,"Time"]
    lambda_eff = OpticalData.loc[i,"Wavelength"]*10
    flux = OpticalData.loc[i,"F_o"]/1e6
    fluxpos = fluxneg = OpticalData.loc[i,"e_F_o"]/1e6
    if fluxneg == 0:
        fluxneg = np.inf
    newrow = {'GRB':GRB, 'Observatory':obs, 'Instrument':inst,'Filter':filt, 'λ_eff':lambda_eff,
              'Time (s)':time, 'Source':'Fong', 'Flux (Jy)':a_u(flux,fluxpos,fluxneg)}
    all_optical = pd.concat([all_optical, pd.DataFrame([newrow])], ignore_index=True)
    
all_optical.sort_values(by=["GRB","Time (s)"],ascending=[False,True],inplace=True)
all_optical.reset_index(inplace=True,drop=True)

In [ ]:
print("Finished optical.")

## Information compilation

### Match data in time

In [ ]:
# max_dt (allowable % time difference) is set before this notebook is run
log_mean_energy = 10**np.mean((np.log10(0.3),np.log10(10)))
nu_x = a_u(log_mean_energy,10-log_mean_energy,log_mean_energy-0.3) * 241797944177033445 # xray frequency [Hz]
results = pd.DataFrame(columns=["GRB","t_o","dt%","nu_o","F_o","nu_x","F_x","B_ox"])
for i_o in all_optical.index: # for each optical data point
    t_o = all_optical.loc[i_o,"Time (s)"] # optical observation time
    F_o = all_optical.loc[i_o,"Flux (Jy)"] # optical flux
    nu_o = 299792458/float(all_optical.loc[i_o,"λ_eff"]/1e10) # optical frequency [Hz]
    for i_x in xrt_data[xrt_data["GRB"]==all_optical.loc[i_o,"GRB"]].index: # for each x-ray data point on this GRB
        t_x = float(xrt_data.loc[i_x,"Time"]) # x-ray observation time
        dt = np.abs(t_o-t_x)/t_x # time difference
        if dt <= max_dt: # if time separation is allowable
            F_x = xrt_data.loc[i_x,"SpecFlux"]
            Beta_ox = -np.log10(F_x/F_o)/np.log10(nu_x/nu_o)
            if pd.notna(Beta_ox.value):
                match_info = {"GRB":all_optical.loc[i_o,"GRB"], "t_o":t_o, "dt%":dt,
                              "nu_o":nu_o, "F_o":F_o, "nu_x":nu_x, "F_x":F_x, "B_ox":Beta_ox}
                results = pd.concat([results, pd.DataFrame([match_info])], ignore_index=True)
        else: # if data points don't match
            pass

### Incorporate error in $\beta_\text{ox}$ due to temporal separation ($\Delta\beta_\text{ox}$)

In [ ]:
delta_B_ox = lambda dt, alpha=1 : np.abs(alpha*np.log10(1+dt)) # Fitzpatrick Eq. 41

if error_B_ox: # flag set before running
    results["α"] = np.nan
    for (grb,t_o),matches in results.groupby(["GRB","t_o"]):
        i = matches.index[0]
        a = results.loc[i,"α"]
        if pd.notna(a):
            continue
        try:
            a = get_temporalIndex(grb, t_o, sGRBs)
            results.loc[matches.index,"α"] = a.value
        except:
            a = 1
            results.loc[matches.index,"α"] = a
        print("", end="\rWorking on GRB {}...".format(grb+" "*(7-len(grb))))
    print("\rFinished Δβₒₓ."+" "*11)
    results["B_ox_w_err"] = [match["B_ox"].add_error(delta_B_ox(match["dt%"], match["α"])) for _, match in results.iterrows()]
    B_ox_name = "B_ox_w_err"
    
else:
    B_ox_name = "B_ox"
    
print("Using",B_ox_name)

### Compare with $\beta_\text{x}$ to determine darkness

In [ ]:
results = results.merge(sGRBs[["GRB","Beta_X","Beta_X_pos","Beta_X_neg"]],
                        on="GRB",how="left") # match by GRB ID
results["B_x"] = [a_u(*results.loc[i,["Beta_X","Beta_X_pos","Beta_X_neg"]]) for i in results.index] # construct objects
results.drop(['Beta_X', 'Beta_X_pos', 'Beta_X_neg'],axis=1,inplace=True) # discard superfluous columns

# add flag columns for darkness by both methods
for i in results.index:
    if restrictive: # flag set before running. '<<' operator is overloaded for use with a_u class
        results.loc[i, "Jak_dark"] = (results.loc[i, B_ox_name] << 0.5)
        results.loc[i, "vdH_dark"] = (results.loc[i, B_ox_name] << results.loc[i,"B_x"]-0.5)
    else:
        results.loc[i, "Jak_dark"] = (results.loc[i, B_ox_name] < 0.5)
        results.loc[i, "vdH_dark"] = (results.loc[i, B_ox_name] < results.loc[i,"B_x"]-0.5)

# results.sort_values(by=["GRB","t_o"],ascending=[False,True])

In [ ]:
dark_matches = results.loc[(results["Jak_dark"] | results["vdH_dark"]), :].sort_values(by=["GRB","dt%"], ascending=[False,True])

# only accept the closest temporal match for each optical data point
close_times = results.copy()
for grb_id in close_times["GRB"].unique():
    working = close_times[close_times["GRB"]==grb_id]
    for optical_point in working["t_o"].unique():
        subworking = working[working["t_o"]==optical_point]
        closest = subworking["dt%"].min()
        close_times.drop(subworking[subworking["dt%"] != closest].index, axis=0, inplace=True)

# darkest matched pair for each GRB (not necessarily *dark*)
darkest_times = results.copy()
for grb_id,working in darkest_times.groupby("GRB"):
    if any([np.isfinite(fx.minus) for fx in working["F_x"]]):
        darkest_times.drop(working[[np.isinf(fx.minus) for fx in working["F_x"]]].index,axis=0,inplace=True)
        working = working[[np.isfinite(fx.minus) for fx in working["F_x"]]]
    darkest_beta = working["B_ox"].min()
    darkest_times.drop(working[working["B_ox"] != darkest_beta].index, axis=0, inplace=True)
    
for GRB in close_times["GRB"].unique():
    print(GRB+"\t",close_times["GRB"].tolist().count(GRB))